In [ ]:
import os

import fsspec

from utils.hls import HLSBand
from utils.hls import HLSCatalog
from utils.hls import HLSTileLookup
from utils.hls import fia_csv_to_data_catalog_input

In [2]:
lookup = HLSTileLookup()

Reading tile extents...
Read tile extents for 56686 tiles


In [3]:
bands = [
    HLSBand.COASTAL_AEROSOL,
    HLSBand.BLUE,
    HLSBand.GREEN,
    HLSBand.RED,
    HLSBand.NIR_NARROW,
    HLSBand.SWIR1,
    HLSBand.SWIR2,
    HLSBand.CIRRUS,
    HLSBand.QA,
]

In [4]:
point_catalog = HLSCatalog.from_point_pandas(df=fia_csv_to_data_catalog_input('./fia_10.csv'), bands=bands, tile_lookup=lookup)

In [26]:
os.environ['AZURE_ACCOUNT_KEY'] = "{YOUR_STORAGE_ACCOUNT_KEY}"

In [21]:
storage_container = 'fia'
account_name = 'usfs'
account_key = os.environ['AZURE_ACCOUNT_KEY']

In [ ]:
path = 'catalogs/fia10'
write_store = fsspec.get_mapper(
    f"az://{storage_container}/{path}.zarr",
    account_name=account_name,
    account_key=account_key
)
point_catalog.to_zarr(write_store)

In [6]:
pc = HLSCatalog.from_zarr('fia10.zarr')

In [12]:
%%time

# 2015-2019 USA
bbox = [-124.98046874999999, 24.367113562651262, -66.70898437499999, 49.49667452747045]
years = [2015, 2016, 2017, 2018, 2019]
bbox_catalog = HLSCatalog.from_bbox(bbox, years, bands, lookup)

CPU times: user 46min 10s, sys: 16.6 s, total: 46min 27s
Wall time: 1h 11min 53s


In [23]:
path = 'catalogs/hls_conus_2015-2019'
write_store = fsspec.get_mapper(
    f"az://{storage_container}/{path}.zarr",
    account_name=account_name,
    account_key=account_key
)
bbox_catalog.to_zarr(write_store)